In [1]:
from IPython.display import display, HTML, clear_output
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import mlflow
import predict as predict_script
import deploy_model as dm

import sys, os
sys.path.append(os.path.abspath('..'))

In [3]:
import utils as ut

In [6]:
import numerapi
napi = numerapi.NumerAPI(verbosity="info")

In [7]:
import logging
logging.basicConfig(level=logging.INFO)

In [8]:
import os


TEST = True
TMP_DOWNLOADS_DIR = "./downloads/"
MODELS_DIR = "./models/"
AWS_CREDS_FL = "~/.aws/personal_credentials"
RUN_COLS = ["run_id", "experiment_id", "artifact_uri", "metrics.sharpe", "params.target", "params.lgbm_params.num_leaves"]
EXPERIMENT_NAME = "all_data_v2"

os.makedirs(TMP_DOWNLOADS_DIR, exist_ok=True)
os.makedirs(MODELS_DIR, exist_ok=True)

## 1. Sunshine model trained on all data

In [28]:
SUNSHINE_MODEL_NAME = "albania"

### 1.1 Download the model

In [10]:
mlflow.set_tracking_uri("http://18.218.213.146:5500")

In [11]:
experiment_id = mlflow.get_experiment_by_name(EXPERIMENT_NAME).experiment_id
experiment_id

'17'

In [12]:
run_df = mlflow.search_runs(experiment_id)
run_df.columns

Index(['run_id', 'experiment_id', 'status', 'artifact_uri', 'start_time',
       'end_time', 'metrics.mean', 'metrics.max_drawdown', 'metrics.apy',
       'metrics.std', 'metrics.sharpe', 'params.lgbm_params.learning_rate',
       'params.train_ident', 'params.target', 'params.cv',
       'params.lgbm_params.max_depth', 'params.embargo', 'params.impute',
       'params.lgbm_params.colsample_bytree', 'params.lgbm_params.n_jobs',
       'params.int8', 'params.lgbm_setting', 'params.lgbm_params.num_leaves',
       'params.ensembling', 'params.expt_name', 'params.targets',
       'params.sample_every_nth', 'params.lgbm_params.n_estimators',
       'params.features', 'tags.mlflow.user', 'tags.mlflow.source.name',
       'tags.mlflow.runName', 'tags.mlflow.source.type',
       'tags.mlflow.log-model.history'],
      dtype='object')

In [13]:
run_df[RUN_COLS]

,run_id,experiment_id,artifact_uri,metrics.sharpe,params.target,params.lgbm_params.num_leaves
0,375735810ae646c0a48c15c70fa06168,17,s3://numerai-v1/mlflow/17/375735810ae646c0a48c...,9.678721,target_waldo_v4_20,64
1,0c005067614a41808edf9791be0785a4,17,s3://numerai-v1/mlflow/17/0c005067614a41808edf...,12.174212,target_victor_v4_20,64
2,d2414f840f844f68a08b1650087cbf8b,17,s3://numerai-v1/mlflow/17/d2414f840f844f68a08b...,9.093966,target_tyler_v4_20,64
3,3a4a80ecf965404bb958bf70f9e89423,17,s3://numerai-v1/mlflow/17/3a4a80ecf965404bb958...,9.531144,target_ralph_v4_20,64
4,c8febcc8be214109a32ecd02175614ca,17,s3://numerai-v1/mlflow/17/c8febcc8be214109a32e...,4.873786,target_jerome_v4_60,64
5,2ac89fcadef4422b83f39457409c1d88,17,s3://numerai-v1/mlflow/17/2ac89fcadef4422b83f3...,9.886836,target_nomi_v4_20,64


In [14]:
ut.download_from_s3_recursively(
    s3_path=f"s3://numerai-v1/mlflow/{experiment_id}/",
    local_path=TMP_DOWNLOADS_DIR,
    aws_credential_fl=AWS_CREDS_FL,
    dry_run=True,
    flname="model.pkl",
)

[2023-04-21 16:13:48]  Loading aws credenitals from ~/.aws/personal_credentials...
[2023-04-21 16:13:48]  [DRYRUN] Would download s3://numerai-v1/mlflow/17/0c005067614a41808edf9791be0785a4/artifacts/all_data_traintest_target_victor_v4_20_all_data_ff2c9e/model.pkl to ./downloads/0c005067614a41808edf9791be0785a4/artifacts/all_data_traintest_target_victor_v4_20_all_data_ff2c9e/model.pkl
[2023-04-21 16:13:48]  [DRYRUN] Would download s3://numerai-v1/mlflow/17/2ac89fcadef4422b83f39457409c1d88/artifacts/all_data_traintest_target_nomi_v4_20_all_data_ff2c9e/model.pkl to ./downloads/2ac89fcadef4422b83f39457409c1d88/artifacts/all_data_traintest_target_nomi_v4_20_all_data_ff2c9e/model.pkl
[2023-04-21 16:13:48]  [DRYRUN] Would download s3://numerai-v1/mlflow/17/375735810ae646c0a48c15c70fa06168/artifacts/all_data_traintest_target_waldo_v4_20_all_data_ff2c9e/model.pkl to ./downloads/375735810ae646c0a48c15c70fa06168/artifacts/all_data_traintest_target_waldo_v4_20_all_data_ff2c9e/model.pkl
[2023-04-21

In [15]:
if not TEST:
    ut.download_from_s3_recursively(
        s3_path=f"s3://numerai-v1/mlflow/{experiment_id}/",
        local_path=TMP_DOWNLOADS_DIR,
        aws_credential_fl=AWS_CREDS_FL,
        flname="model.pkl",
    )

[2023-04-21 16:13:54]  Loading aws credenitals from ~/.aws/personal_credentials...
[2023-04-21 16:13:54]  Downloading s3://numerai-v1/mlflow/17/0c005067614a41808edf9791be0785a4/artifacts/all_data_traintest_target_victor_v4_20_all_data_ff2c9e/model.pkl to ./downloads/0c005067614a41808edf9791be0785a4/artifacts/all_data_traintest_target_victor_v4_20_all_data_ff2c9e/model.pkl
[2023-04-21 16:14:13]  Downloading s3://numerai-v1/mlflow/17/2ac89fcadef4422b83f39457409c1d88/artifacts/all_data_traintest_target_nomi_v4_20_all_data_ff2c9e/model.pkl to ./downloads/2ac89fcadef4422b83f39457409c1d88/artifacts/all_data_traintest_target_nomi_v4_20_all_data_ff2c9e/model.pkl
[2023-04-21 16:14:31]  Downloading s3://numerai-v1/mlflow/17/375735810ae646c0a48c15c70fa06168/artifacts/all_data_traintest_target_waldo_v4_20_all_data_ff2c9e/model.pkl to ./downloads/375735810ae646c0a48c15c70fa06168/artifacts/all_data_traintest_target_waldo_v4_20_all_data_ff2c9e/model.pkl
[2023-04-21 16:14:49]  Downloading s3://numerai

In [16]:
def extract_target_name(model_path, suffix):
    # ./downloads/d2414f840f844f68a08b1650087cbf8b/artifacts/all_data_traintest_target_tyler_v4_20_all_data_ff2c9e/model.pkl -> tyler_v4_20
    return model_path[model_path.find("target")+7:].split("/")[0][:-len(suffix)-7]

extract_target_name("./downloads/d2414f840f844f68a08b1650087cbf8b/artifacts/all_data_traintest_target_tyler_v4_20_all_data_ff2c9e/model.pkl", suffix="_all_data")

'tyler_v4_20'

In [17]:
# load the models up into a dict {"model_{target_name}": model} where target name
# is the name is extracted from the model's path.
# ./downloads/d2414f840f844f68a08b1650087cbf8b/artifacts/all_data_traintest_target_tyler_v4_20_all_data_ff2c9e/model.pkl -> tyler_v4_20
# Iterate over all files in this folder recursively
models = {}
for root, dirs, files in os.walk(TMP_DOWNLOADS_DIR):
    for file in files:
        if file != "model.pkl":
            continue
        model_path = os.path.join(root, file)
        print(model_path)
        target_name = extract_target_name(model_path, suffix="_all_data")
        models[f"model_{target_name}"] = ut.unpickle_obj(model_path)

./downloads/375735810ae646c0a48c15c70fa06168/artifacts/all_data_traintest_target_waldo_v4_20_all_data_ff2c9e/model.pkl
./downloads/3a4a80ecf965404bb958bf70f9e89423/artifacts/all_data_traintest_target_ralph_v4_20_all_data_ff2c9e/model.pkl
./downloads/0c005067614a41808edf9791be0785a4/artifacts/all_data_traintest_target_victor_v4_20_all_data_ff2c9e/model.pkl
./downloads/d2414f840f844f68a08b1650087cbf8b/artifacts/all_data_traintest_target_tyler_v4_20_all_data_ff2c9e/model.pkl
./downloads/c8febcc8be214109a32ecd02175614ca/artifacts/all_data_traintest_target_jerome_v4_60_all_data_ff2c9e/model.pkl
./downloads/2ac89fcadef4422b83f39457409c1d88/artifacts/all_data_traintest_target_nomi_v4_20_all_data_ff2c9e/model.pkl


### 1.2 Create the ensemble model and test it

In [18]:
ensemble_model = dm.MultiTargetNeutralModel(models=models)

In [19]:
predict_script.predict(napi=napi, wrapped_model=ensemble_model)

v4.1/live.parquet: 4.49MB [00:01, 3.23MB/s]                                                                                                                                                                         
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:03<00:00,  3.23s/it]


,prediction
id,
n00069c7a1b739b3,0.762442
n0007dc4d31340ae,0.736853
n000fb086d5b9090,0.158573
n00243ffdd54dbac,0.244409
n00257130c83832f,0.954866
...,...
nffde3a39aafb293,0.919202
nffe4290388bd16c,0.452750
nfff050628794d66,0.479146


In [29]:
ss_fl = os.path.join(MODELS_DIR, SUNSHINE_MODEL_NAME+".pkl")
ut.pickle_obj(fl=ss_fl, obj=ensemble_model)
print(ss_fl)

./models/albania.pkl


In [31]:
ut.upload_to_s3_recursively(
    s3_path="s3://numerai-v1/deployed_models/",
    dir_path="./models/",
    aws_credential_fl=AWS_CREDS_FL,
    dry_run=False,
)

[2023-04-21 16:38:03]  Loading aws credenitals from ~/.aws/personal_credentials...
[2023-04-21 16:38:03]  Uploading ./models/albania.pkl to s3://numerai-v1/deployed_models/albania.pkl
[2023-04-21 16:41:37]  Uploading ./models/sunshine_6tgts_mean_n50.pkl to s3://numerai-v1/deployed_models/sunshine_6tgts_mean_n50.pkl
